In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder,OrdinalEncoder,FunctionTransformer
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import keras_tuner as kt
warnings.filterwarnings('ignore')
%matplotlib inline

In [1]:
!pip install -q -U keras-tuner

DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\aiofiles-23.2.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\anyio-3.7.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\boto3-1.26.105-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anac

In [4]:
df=pd.read_csv('gemstone.csv')

In [5]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [6]:
df.drop(labels=['id'],axis=1)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [23]:
X=df.drop(labels=['price','id'],axis=1)
y=df[['price']]

In [24]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [25]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [26]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [27]:
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
168192,0.34,Ideal,I,VVS2,60.9,57.0,4.56,4.53,2.76
35202,0.90,Good,E,SI1,63.8,57.0,6.07,6.03,3.87
41091,1.02,Premium,G,VS1,62.7,58.0,6.35,6.39,4.00
31239,0.32,Premium,G,VS2,62.1,59.0,4.37,4.35,2.71
45722,0.35,Ideal,J,VVS2,61.1,56.0,4.53,4.57,2.78


In [28]:
X_train.shape

(135501, 9)

In [29]:
X_test.shape

(58072, 9)

In [30]:
num_col=X.select_dtypes(exclude='object').columns
cat_col=X.select_dtypes(include='object').columns

In [31]:
num_col

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [15]:
num_col_indices = [X.columns.get_loc(col) for col in X.select_dtypes(exclude='object').columns]

cat_col_indices = [X.columns.get_loc(col) for col in X.select_dtypes(include='object').columns]

In [16]:
num_col_indices

[0, 1, 5, 6, 7, 8, 9]

In [17]:
cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
clarity_map = {"I1":1,"SI2":2 ,"SI1":3 ,"VS2":4 , "VS1":5 , "VVS2":6 , "VVS1":7 ,"IF":8}
color_map = {"D":1 ,"E":2 ,"F":3 , "G":4 ,"H":5 , "I":6, "J":7}

In [18]:
cut_cat=list(cut_map.keys())
clarity_cat=list(clarity_map.keys())
color_cat=list(color_map.keys())

In [38]:
def remove_outlier(X,col):
    X_fil=X.copy()
    for i in col:
        iqr=(X[i].quantile(0.75)-X[i].quantile(0.25))
        u_lt=X[i].quantile(0.75) + 1.5*iqr
        l_lt=X[i].quantile(0.25) -1.5*iqr
        X_fil[i]=np.where(
            X_fil[i]>u_lt,
            u_lt,
            np.where(
                X_fil[i]<l_lt,
                l_lt,
                X_fil[i]
            )
        )
    return X_fil

In [58]:
from sklearn.base import TransformerMixin, BaseEstimator

class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X_fil = X.copy()
            for i in self.col:
                iqr = (X_fil[i].quantile(0.75) - X_fil[i].quantile(0.25))
                u_lt = X_fil[i].quantile(0.75) + 1.5 * iqr
                l_lt = X_fil[i].quantile(0.25) - 1.5 * iqr
                X_fil[i] = np.where(
                    X_fil[i] > u_lt,
                    u_lt,
                    np.where(
                        X_fil[i] < l_lt,
                        l_lt,
                        X_fil[i]
                    )
                )
            return X_fil
        elif isinstance(X, np.ndarray):
            X_fil = X.copy()
            for i, col_name in enumerate(self.col):
                iqr = np.percentile(X_fil[:, i], 75) - np.percentile(X_fil[:, i], 25)
                u_lt = np.percentile(X_fil[:, i], 75) + 1.5 * iqr
                l_lt = np.percentile(X_fil[:, i], 25) - 1.5 * iqr
                X_fil[:, i] = np.where(
                    X_fil[:, i] > u_lt,
                    u_lt,
                    np.where(
                        X_fil[:, i] < l_lt,
                        l_lt,
                        X_fil[:, i]
                    )
                )
            return X_fil
        else:
            raise ValueError("Input must be a DataFrame or numpy array")

    def get_feature_names_out(self, input_features=num_col):
           return input_features


In [59]:
# numerical pipeline
num_pip=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler()),
    ('outlier_remove',OutlierRemover(col=num_col))
    ]

)

# catagorical pipeline
cat_pip=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_cat,color_cat,clarity_cat])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pip,num_col),
('cat_pipeline',cat_pip,cat_col)
])


In [60]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [62]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135501 entries, 0 to 135500
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   num_pipeline__carat    135501 non-null  float64
 1   num_pipeline__depth    135501 non-null  float64
 2   num_pipeline__table    135501 non-null  float64
 3   num_pipeline__x        135501 non-null  float64
 4   num_pipeline__y        135501 non-null  float64
 5   num_pipeline__z        135501 non-null  float64
 6   cat_pipeline__cut      135501 non-null  float64
 7   cat_pipeline__color    135501 non-null  float64
 8   cat_pipeline__clarity  135501 non-null  float64
dtypes: float64(9)
memory usage: 9.3 MB


In [73]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58072 entries, 0 to 58071
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num_pipeline__carat    58072 non-null  float64
 1   num_pipeline__depth    58072 non-null  float64
 2   num_pipeline__table    58072 non-null  float64
 3   num_pipeline__x        58072 non-null  float64
 4   num_pipeline__y        58072 non-null  float64
 5   num_pipeline__z        58072 non-null  float64
 6   cat_pipeline__cut      58072 non-null  float64
 7   cat_pipeline__color    58072 non-null  float64
 8   cat_pipeline__clarity  58072 non-null  float64
dtypes: float64(9)
memory usage: 4.0 MB


In [74]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [64]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]
cross_val_scores_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)
    score=np.mean(cross_val_score(model,X_train,y_train,cv=5,scoring='r2'))

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)
    print("cross_val_score",score*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 993.864471640182
MAE: 664.6696914908536
R2 score 93.93609268000924
cross_val_score 93.8777341983692


Lasso
Model Training Performance
RMSE: 994.0403878292017
MAE: 665.1103528578799
R2 score 93.93394584026528
cross_val_score 93.88007101808157


Ridge
Model Training Performance
RMSE: 993.865730405243
MAE: 664.688727003093
R2 score 93.93607731968655
cross_val_score 93.87784485511158


Elasticnet
Model Training Performance
RMSE: 1557.0210266026943
MAE: 1077.065034111917
R2 score 85.11711320921417
cross_val_score 85.05017201365949




In [65]:
!pip install tensorflow

DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\aiofiles-23.2.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\anyio-3.7.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\boto3-1.26.105-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anac

In [83]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\aiofiles-23.2.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\anyio-3.7.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\boto3-1.26.105-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anac

In [85]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

In [86]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)  
])

In [87]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [88]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 3s 648us/step - loss: 11353665.0000 - val_loss: 698882.3125
Epoch 2/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 627us/step - loss: 849651.1875 - val_loss: 635289.8750
Epoch 3/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 625us/step - loss: 804569.0000 - val_loss: 580829.4375
Epoch 4/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 619us/step - loss: 733840.0000 - val_loss: 532724.6875
Epoch 5/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 630us/step - loss: 683556.1875 - val_loss: 507000.4062
Epoch 6/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 619us/step - loss: 657658.8750 - val_loss: 495979.0312
Epoch 7/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 626us/step - loss: 639830.7500 - val_loss: 488636.0938
Epoch 8/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 637us/step - loss: 628734.6250 - val_loss: 487233.2188
Epoch 9/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 624us/step - loss: 617190.0000 - val_loss: 482108.1562
Epoch 10/100
3388/3388 ━━━━━━━━━━━━━━━━━━━━ 2s 654us/step - loss: 621664.5000 - val_loss:

In [89]:
y_pred = model.predict(X_test)

mae, rmse, r2_square = evaluate_model(y_test, y_pred)
# score=np.mean(cross_val_score(model,X_train,y_train,cv=5,scoring='r2'))

print('Model Test Performance')
print("MAE:", mae)
print("RMSE:", rmse)
print("R2 score:", r2_square * 100)
# print("cross_val_score",score*100)

1815/1815 ━━━━━━━━━━━━━━━━━━━━ 1s 396us/step
Model Test Performance
MAE: 472.43158620289654
RMSE: 663.6575312818873
R2 score: 97.29612707117278


In [91]:
models['Neural Network']=model

In [92]:
model_list.append('Neural Network')

In [93]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet', 'Neural Network']